# Les identifiants

In [1]:
%%shell
(cd samples/hellojpa && mvn --quiet clean package )

In [72]:
%%shell
docker start H2-JPADemo || docker run --quiet --detach --rm --name=H2-JPADemo \
    -e PUID=$UID -e PGID=$GID \
    -e H2_ARGS="-tcp -tcpAllowOthers -ifNotExists" \
    -p 9092:9092 \
    -v h2-data:/opt/h2-data \
    brunoe/docker-database-h2:develop

H2-JPADemo


In [2]:
%jars samples/hellojpa/target/*-SNAPSHOT-jar-with-dependencies.jar

import jakarta.persistence.*;
import fr.univtln.bruno.demos.jpa.hello.DatabaseManager;
System.setProperty("jakarta.persistence.jdbc.url","jdbc:h2:tcp://host.docker.internal/hellojpa-db");

try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) { }

18:11:22.757 INFO  f.u.b.d.jpa.hello.DatabaseManager - Override JPA config jakarta.persistence.jdbc.url with system property jakarta.persistence.jdbc.url
18:11:22.762 INFO  f.u.b.d.jpa.hello.DatabaseManager - Override JPA config jakarta.persistence.jdbc.user with system property jakarta.persistence.jdbc.user
18:11:22.765 INFO  f.u.b.d.jpa.hello.DatabaseManager - Override JPA config jakarta.persistence.jdbc.password with system property jakarta.persistence.jdbc.password
18:11:24.649 INFO  com.mchange.v2.log.MLog - MLog clients using slf4j logging.
18:11:24.878 INFO  SQL dialect - HHH000400: Using dialect: org.hibernate.dialect.H2Dialect


In [3]:
%maven net.datafaker:datafaker:1.8.0
import net.datafaker.Faker;

In [4]:
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

Logger log = LoggerFactory.getLogger("Notebook");

## UUIDs

In [4]:
%%javasrcClassByName Customer
samples/hellojpa/src/main/java/fr/univtln/bruno/demos/jpa/hello/samples/ex_uuid/Customer.java

```Java
@Entity
@Data
@RequiredArgsConstructor(staticName = "of")
@NoArgsConstructor(access = AccessLevel.PROTECTED)
@Table(name = "CUSTOMER", schema = "EX_UUID")
public class Customer {

    @Id
    @GeneratedValue(strategy = GenerationType.UUID)
    private UUID uuid;

    @Column(length = 50, nullable = false)
    @NonNull
    private String name;
}
```

In [5]:
import java.util.stream.Stream;

import fr.univtln.bruno.demos.jpa.hello.samples.ex_UUID.Customer;

try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {
            entityManager.getTransaction().begin();

            Faker faker = new Faker();

            Stream.generate(() -> {
                String fullname = faker.name().fullName();
                return Customer.of(fullname);
            }).limit(5).forEach(entityManager::persist);

            entityManager.getTransaction().commit();

            entityManager.createQuery("""
                    SELECT c 
                    FROM CustomerUUID c""")
                    .setMaxResults(5)
                    .getResultStream()
                    .forEach(System.out::println);
        }

Customer(uuid=6c8f49c3-ab06-46d3-b3c4-afffa5dce270, name=Faye Steuber Jr.)
Customer(uuid=7b45f312-44ab-4b46-8c07-f9f5e5f85eff, name=Donald Romaguera III)
Customer(uuid=f61a0da6-3300-4521-9298-d69afa27faad, name=Jordon Huel)
Customer(uuid=ffa3af0a-811b-4695-8788-8cd1e6a89816, name=Evan Douglas)
Customer(uuid=9d608bb4-3713-4d3f-8ea4-18771adc23ec, name=Thalia Boyle)
